<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Connect/blob/main/Low_Beta_Industry_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the required packages
import itertools
from AlgorithmImports import *

class LowBetaPortfolio(QCAlgorithm):

    def initialize(self):
        self.set_start_date(2015, 1, 1) # Define the start date
        self.set_end_date(2025,1,1) # Define the end date
        self.set_cash("USD", 10000000) # Set the cash allocated to the strategy
        self.settings.minimum_order_margin_portfolio_percentage = 0.005 # Define the minimum order margin percentage as 0.5%
        self.settings.automatic_indicator_warm_up = True # Provide a warm up period for the algorithm
        self.set_security_initializer(BrokerageModelSecurityInitializer(self.brokerage_model, FuncSecuritySeeder(self.get_last_known_prices))) # Get the last known prices using the Brokerage model
        self._assets_per_industry = self.get_parameter('assets_per_industry', 30) # Define the number of stocks that will be selected from each industry, in this case it will be 30
        self._beta_period = self.get_parameter('beta_period', 90) # Define the period to estimate our betas over, in this case it will be 90 days
        self._spy = self.add_equity('SPY', Resolution.DAILY) # Define the index (the S&P 500) from which we will use for our strategy
        self.universe_settings.resolution = Resolution.DAILY # Set the resolution period to daily
        self.universe_settings.schedule.on(self.date_rules.month_start(self._spy.symbol))  # Carry out the selection process at the start of each month and form the portfolio
        self.add_universe(self.stock_selection)
        self.schedule.on(self.date_rules.month_start(self._spy.symbol, 2), self.time_rules.midnight, self.rebalance_portfolio) # Rebalance the portfolio at midnight at the end of each month

    def rebalance_portfolio(self):
        if self._targets:
            # Rebalance the portfolio.
            self.set_holdings(self._targets)
            self._targets = []

    def beta(self, tickers):
        rets = self.history([self._spy.symbol] + tickers, self._beta_period, Resolution.DAILY, fill_forward=False).close.unstack(0).dropna(axis=1).pct_change().dropna()
        symbols = [s for s in tickers if s in rets.columns]
        df_reg = rets[[self._spy.symbol] + symbols]
        # The dependent variable is the market return
        x = df_reg.values[:, [0]]
        # Concatenate a column of ones to act for the intercept
        x = np.concatenate([np.ones_like(x), x], axis=1)
        # Determine the betas
        betas = np.linalg.pinv(x.T.dot(x)).dot(x.T).dot(df_reg.values[:, 1:])
        return pd.Series(betas[1], df_reg.columns[1:], name='Beta')

    def stock_selection(self, stocks):
        # From each industry group select the most liquid stocks
        sorted_stocks = sorted([x for x in stocks if x.asset_classification.morningstar_industry_group_code], key=lambda x: (x.asset_classification.morningstar_industry_group_code, x.dollar_volume))
        selected_stocks = []
        for _, fundamentals_by_group in itertools.groupby(sorted_stocks, lambda x: x.asset_classification.morningstar_industry_group_code):
            selected_stocks.extend(list(fundamentals_by_group)[-self._assets_per_industry:])
        # Determine the absolute beta for each stock
        self.beta_by_symbol = self.beta([x.symbol for x in selected_stocks]).abs()
        # Determine the median beta
        med_beta = np.median(self.beta_by_symbol.values)
        # For each industry select stocks in which the beta is less than the median for the group
        allocation_by_industry = {}
        tickers = []
        for industry_ticker, industry_stocks in itertools.groupby(selected_stocks, lambda x: x.asset_classification.morningstar_industry_group_code):
            # Get the beta of each asset in the industry.
            industry_beta_by_symbol = self.beta_by_symbol[[x.symbol for x in industry_stocks if x.symbol in self.beta_by_symbol]]
            # Get the betas for stocks that are below the median
            low_betas = industry_beta_by_symbol[industry_beta_by_symbol < med_beta]
            if low_betas.empty:
                continue
            tickers.extend(list(low_betas.index))
            # Create weights for the assets in this industry (Beta-weighted weights).
            beta_ranks = low_betas.sort_values().rank(method='first', ascending=False)
            allocation_by_industry[industry_ticker] = beta_ranks / beta_ranks.sum()

        # Create the portfolio targets. Give equal weight to each industry. Liquidate assets we no longer want.
        self._targets = [PortfolioTarget(symbol, 0) for symbol, holding in self.portfolio.items() if holding.invested and symbol not in tickers]
        for industry_stocks in allocation_by_industry.values():
            self._targets.extend([PortfolioTarget(symbol, weight/len(allocation_by_industry)) for symbol, weight in industry_stocks.items()])
        return tickers
